In [1]:
# Adjust Python path within the notebook
import sys
project_root = '/Users/thangnguyen/Documents/GitHub/project-1-individual-knam2609'
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import scripts

In [2]:
import matplotlib.pyplot as plt
import seaborn as sb
import pandas
import os

In [3]:
# Create SparkSession
spark = scripts.clean_base.create_spark_session()

24/08/27 17:10:34 WARN Utils: Your hostname, THANGs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 100.86.89.69 instead (on interface en0)
24/08/27 17:10:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/27 17:10:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Create directories for plots
scripts.download.make_directories(["../plots/"], ["yellow"])

In [5]:
# Create directories for plots
scripts.download.make_directories(["../plots/yellow/"], ["correlation", "histogram", "daily", "hourly"])

In [6]:
yellow_files = scripts.clean_base.list_parquet_directories("../data/raw/yellow/")
yellow_dfs = [spark.read.parquet(file) for file in yellow_files]

In [7]:
yellow_files

['../data/raw/yellow/2023-08.parquet',
 '../data/raw/yellow/2023-11.parquet',
 '../data/raw/yellow/2023-10.parquet',
 '../data/raw/yellow/2023-09.parquet',
 '../data/raw/yellow/2023-06.parquet',
 '../data/raw/yellow/2023-07.parquet']

In [8]:
# Extract the schema from the existing DataFrame
schema = yellow_dfs[0].schema

# Create an empty DataFrame using the extracted schema
unioned_df = spark.createDataFrame([], schema)

unioned_df # merge sub-sample of every datasets

VendorID,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,trip_time,fare_per_miles


In [9]:
# Plotting correlation heatmaps of every datasets
for df in yellow_dfs:
    file_path = yellow_files[yellow_dfs.index(df)]
    # Extract the base name of the file
    file_name = os.path.basename(file_path)
    # Remove the file extension to get only the date part
    date_part = os.path.splitext(file_name)[0]
    print(date_part)
    scripts.manipulate_data.find_min_max_df(df, scripts.clean_yellow.COLUMNS).show()
    print(df.count())
    df = scripts.clean_base.sampling_data(df, "VendorID", 0.1)
    print(df.count())
    unioned_df = unioned_df.union(df)
    scripts.plot_data.plot_correlation_heatmap(df, scripts.clean_yellow.COLUMNS, "yellow", "../plots/yellow/correlation/", date_part)

2023-08


+--------------------+--------------------+------------------+
|              Column|                 Min|               Max|
+--------------------+--------------------+------------------+
|     passenger_count|                 1.0|               9.0|
|       trip_distance|                0.01|            373.51|
|         fare_amount|                 0.0|            1375.0|
|               extra|                 0.0|             14.25|
|             mta_tax|                 0.0|               4.0|
|          tip_amount|                 0.0|            234.88|
|        tolls_amount|                 0.0|             87.75|
|improvement_surch...|                 0.0|               1.0|
|        total_amount|                 0.0|           1435.19|
|congestion_surcharge|                 0.0|               2.5|
|         airport_fee|                 0.0|              1.75|
|           trip_time|0.016666666666666666|            749.35|
|      fare_per_miles|                 0.0|38.909090909

24/08/27 17:10:47 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+--------------------+--------------------+------------------+
|              Column|                 Min|               Max|
+--------------------+--------------------+------------------+
|     passenger_count|                 1.0|               9.0|
|       trip_distance|                0.08|            281.76|
|         fare_amount|                 0.0|            1916.8|
|               extra|                 0.0|             14.25|
|             mta_tax|                 0.0|               4.0|
|          tip_amount|                 0.0|             240.0|
|        tolls_amount|                 0.0|             100.0|
|improvement_surch...|                 0.0|               1.0|
|        total_amount|                 0.0|           1939.55|
|congestion_surcharge|                 0.0|               2.5|
|         airport_fee|                 0.0|              1.75|
|           trip_time|0.016666666666666666|            747.55|
|      fare_per_miles|                 0.0|44.734042553

In [10]:
unioned_df.count()

1708004

In [13]:
# Plotting distribution of each continous columns
for col in scripts.clean_yellow.COLUMNS:
    scripts.plot_data.plot_histogram(unioned_df, col, "yellow", "../plots/yellow/histogram/", 50)

0 16
../plots/yellow/histogram/passenger_count_histogram.png


0 63.64
../plots/yellow/histogram/trip_distance_histogram.png
0.0 328.7
../plots/yellow/histogram/fare_amount_histogram.png
0.0 21.75
../plots/yellow/histogram/extra_histogram.png
0.0 10.5
../plots/yellow/histogram/mta_tax_histogram.png
0.0 60.0
../plots/yellow/histogram/tip_amount_histogram.png
0.0 46.05
../plots/yellow/histogram/tolls_amount_histogram.png
0 11.0
../plots/yellow/histogram/improvement_surcharge_histogram.png
0 351.05
../plots/yellow/histogram/total_amount_histogram.png
0.0 12.5
../plots/yellow/histogram/congestion_surcharge_histogram.png
0.0 11.75
../plots/yellow/histogram/airport_fee_histogram.png
0 159.48333333333332
../plots/yellow/histogram/trip_time_histogram.png
0 54.0
../plots/yellow/histogram/fare_per_miles_histogram.png


Plotting continuous columns against time series like days and hours

In [14]:
from pyspark.sql.functions import avg, to_date, hour

# Convert timestamp to date
for_plot_df = unioned_df.withColumn("pickup_date", to_date(unioned_df["pickup_datetime"]))

In [15]:
# Daily
for col in scripts.clean_yellow.COLUMNS:
    # Group by date and calculate average
    average_df = for_plot_df.groupBy("pickup_date").agg(avg(col).alias(col))
    scripts.plot_data.scatter_plot(average_df, "pickup_date", col, "yellow", "../plots/yellow/daily/")

../plots/yellow/daily/pickup_date_vs_passenger_count_histogram.png


../plots/yellow/daily/pickup_date_vs_trip_distance_histogram.png
../plots/yellow/daily/pickup_date_vs_fare_amount_histogram.png
../plots/yellow/daily/pickup_date_vs_extra_histogram.png
../plots/yellow/daily/pickup_date_vs_mta_tax_histogram.png
../plots/yellow/daily/pickup_date_vs_tip_amount_histogram.png
../plots/yellow/daily/pickup_date_vs_tolls_amount_histogram.png
../plots/yellow/daily/pickup_date_vs_improvement_surcharge_histogram.png
../plots/yellow/daily/pickup_date_vs_total_amount_histogram.png
../plots/yellow/daily/pickup_date_vs_congestion_surcharge_histogram.png
../plots/yellow/daily/pickup_date_vs_airport_fee_histogram.png
../plots/yellow/daily/pickup_date_vs_trip_time_histogram.png
../plots/yellow/daily/pickup_date_vs_fare_per_miles_histogram.png


In [16]:
# Hourly
for col in scripts.clean_yellow.COLUMNS:
    # Extract hour from timestamp and group by it
    hourly_avg_df = df.groupBy(hour(df["pickup_datetime"]).alias("hour")).agg(avg(col).alias(col))
    scripts.plot_data.scatter_plot(hourly_avg_df, "hour", col, "yellow", "../plots/yellow/hourly/")

../plots/yellow/hourly/hour_vs_passenger_count_histogram.png
../plots/yellow/hourly/hour_vs_trip_distance_histogram.png
../plots/yellow/hourly/hour_vs_fare_amount_histogram.png
../plots/yellow/hourly/hour_vs_extra_histogram.png
../plots/yellow/hourly/hour_vs_mta_tax_histogram.png
../plots/yellow/hourly/hour_vs_tip_amount_histogram.png
../plots/yellow/hourly/hour_vs_tolls_amount_histogram.png
../plots/yellow/hourly/hour_vs_improvement_surcharge_histogram.png
../plots/yellow/hourly/hour_vs_total_amount_histogram.png
../plots/yellow/hourly/hour_vs_congestion_surcharge_histogram.png
../plots/yellow/hourly/hour_vs_airport_fee_histogram.png
../plots/yellow/hourly/hour_vs_trip_time_histogram.png
../plots/yellow/hourly/hour_vs_fare_per_miles_histogram.png


Compare average values of continuous columns grouped by categorical columns

In [17]:
print("Grouped by: VendorID")
print(scripts.manipulate_data.group_by_and_avg(unioned_df, "VendorID", scripts.clean_yellow.COLUMNS).sort_values("VendorID"))

Grouped by: VendorID


24/08/27 17:12:25 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


   VendorID  average_passenger_count  average_trip_distance  \
0         1                 1.312069               3.299226   
1         2                 1.416706               3.671397   

   average_fare_amount  average_extra  average_mta_tax  average_tip_amount  \
0            18.820487       3.474262         0.496337            3.435488   
1            20.525597       1.054442         0.496892            3.750970   

   average_tolls_amount  average_improvement_surcharge  average_total_amount  \
0              0.541342                       0.999846             27.767762   
1              0.659615                       0.999855             29.993017   

   average_congestion_surcharge  average_airport_fee  average_trip_time  \
0                      2.352291             0.134324          16.619489   
1                      2.325042             0.175706          17.058816   

   average_fare_per_miles  
0               12.869774  
1               12.537151  


In [18]:
print("Grouped by: RateCodeID")
print(scripts.manipulate_data.group_by_and_avg(unioned_df, "RateCodeID", scripts.clean_yellow.COLUMNS).sort_values("RateCodeID"))

Grouped by: RateCodeID


   RateCodeID  average_passenger_count  average_trip_distance  \
1           1                 1.381342               2.834701   
3           2                 1.598835              18.343341   
2           3                 1.702873              18.231540   
4           4                 1.452302              20.786796   
0           5                 1.738462              16.177332   
5           6                 1.000000               1.910000   

   average_fare_amount  average_extra  average_mta_tax  average_tip_amount  \
1            17.344789       1.626313         0.499667            3.282739   
3            69.987555       1.642185         0.499691           11.189631   
2            93.829932       0.699369         0.000088           14.598870   
4           113.288087       1.941742         0.404212           11.970947   
0            93.667134       0.708409         0.053585           10.840183   
5             5.500000       0.000000         0.500000            0.000000  

In [19]:
print("Grouped by: payment_type")
print(scripts.manipulate_data.group_by_and_avg(unioned_df, "payment_type", scripts.clean_yellow.COLUMNS).sort_values("payment_type"))

Grouped by: payment_type


   payment_type  average_passenger_count  average_trip_distance  \
0             1                 1.375165               3.556356   
2             2                 1.476275               3.700738   
1             3                 1.357743               3.530686   
3             4                 1.336254               4.008562   

   average_fare_amount  average_extra  average_mta_tax  average_tip_amount  \
0            20.030567       1.650772         0.496879            4.488600   
2            20.543364       1.482530         0.496578            0.000303   
1            18.854325       2.446852         0.488324            0.005143   
3            21.615002       1.365514         0.491021            0.030379   

   average_tolls_amount  average_improvement_surcharge  average_total_amount  \
0              0.639501                       0.999898             30.244419   
2              0.594279                       0.999822             25.950843   
1              0.567162          

In [20]:
# Write curated dataset
scripts.clean_base.write_data(unioned_df, "../data/curated/yellow/yellow.parquet")